# Calculating GHG Emissions From Electricity Consumption
This notebook uses US EPA eGRID 2021 emission factors to calculate greenhouse gas (GHG) emissions from electricity consumed. The emissions are calculated for each gas (e.g., CO2, CH4) and also calculated based on the zip code where the electricity is consumed. The user will be prompted to enter their five digit zip code and amount of electricity consumed and this notebook will calculate their scope 2 and 3 emissions for each gas.

# Importing Libraries

In [ ]:
import pandas as pd
from tabulate import tabulate

# User Entry
Setting up the user entry. User needs to enter their zip code and amount of energy consumed.


In [ ]:
user_zip = int(input('Enter your five digit zip code:'))
energy_amount = float(input('Enter the amount of energy consumed in MWh:'))

# Emission Factors
The emission factors from EPA eGRID need to be manipulated before performing calculations on the energy amounts.

### Set up
Setting up the eGRID link, column names, GWP values and biomass/fossil lower limits.

In [ ]:
#eGRID 2021 emission factors data
web_link = 'https://www.epa.gov/system/files/documents/2023-01/eGRID2021_summary_tables.xlsx'

In [ ]:
#Specifying new column names for the final emission factor data:
column01 = 'total fossil'
column02 = 'total biomass'
column03 = 'total other'
column04 = 'scope'
column05 = 'co2_factor'
column06 = 'ch4_factor'
column07 = 'n2o_factor'
column08 = 'co2e_factor'
column09 = 'biogenic_co2_factor'

In [ ]:
#GWP Values from IPCC Sixth Assessment Report (AR6)
co2_gwp = 1
ar6_ch4_gwp = 27.9
ar6_ch4_gwp_nonfossil = 27
ar6_ch4_gwp_fossil = 29.8
ar6_n2o_gwp = 273

In [ ]:
#Per the GHG Protocol, biogenic CO2 is treated separately and should be reported outside of scope 1, 2 and 3.
#Biogenic/Fossil Fuel % is used to help decide whether CO2 is CO2 or biogenic CO2 and which AR6 CH4 GWP to use (fossil versus non-fossil)
#These numbers are used in the emission calculation functions below. 
#The lower limit values are currently arbitrary and can be changed based on user preference.
#You can read biomass_lower_limit like "if biomass is less (or more) than x% then it will be CO2 (or biogenic CO2)"
biomass_lower_limit = 0.03
fossil_lower_limit = 0.03

### Merging Table 1 and 2
Loading the data from two tabs of the eGRID emission factor spreadsheet.

In [ ]:
#Gathering the required data from Table 1 and 2
#Table 1
first_sheet_needed = 'Table 1'
first_skip_row_num = 3
first_num_rows = 28
first_original_column_nums_needed = [1, 2, 3, 4, 5, 17]
first_original_column_names_needed = ["eGRID subregion acronym", "eGRID subregion name", "CO2 - Total output emission rates", "CH4 - Total output emission rates", "N2O - Total output emission rates", "Grid Gross Loss (%)"]
#Table 2
second_sheet_needed = 'Table 2'
second_skip_row_num = 2
second_num_rows = 28
second_original_column_nums_needed = [1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
second_original_column_names_needed = ["eGRID subregion acronym", "eGRID subregion name", "Coal", "Oil", "Gas",	"Other Fossil", "Nuclear", "Hydro", "Biomass", "Wind", "Solar",	"Geo- thermal", "Other unknown/ purchased fuel"]

In [ ]:
#Merging the two sheets together for scope 2 factors
df_scope2_1 = pd.read_excel(web_link, sheet_name=first_sheet_needed, skiprows=first_skip_row_num, usecols = first_original_column_nums_needed, nrows = first_num_rows)
df_scope2_1.columns = first_original_column_names_needed
df_scope2_2 = pd.read_excel(web_link, sheet_name=second_sheet_needed, skiprows=second_skip_row_num, usecols = second_original_column_nums_needed, nrows = second_num_rows)
df_scope2_2.columns = second_original_column_names_needed
df_scope2 = df_scope2_1.merge(df_scope2_2, how='left', on="eGRID subregion acronym")

In [ ]:
#Merging the two sheets together for scope 3 factors
df_scope3_1 = pd.read_excel(web_link, sheet_name=first_sheet_needed, skiprows=first_skip_row_num, usecols = first_original_column_nums_needed, nrows = first_num_rows)
df_scope3_1.columns = first_original_column_names_needed
df_scope3_2 = pd.read_excel(web_link, sheet_name=second_sheet_needed, skiprows=second_skip_row_num, usecols = second_original_column_nums_needed, nrows = second_num_rows)
df_scope3_2.columns = second_original_column_names_needed
df_scope3 = df_scope3_1.merge(df_scope3_2, how='left', on="eGRID subregion acronym")

In [ ]:
#Joining Table 1 and 2 together causes a duplicate of one of the columns. 
#Deleting one column and renaming the other column to match the original column name
df_scope2.pop("eGRID subregion name_y")
df_scope2.rename(columns = {'eGRID subregion name_x':'eGRID subregion name'}, inplace = True)
df_scope3.pop("eGRID subregion name_y")
df_scope3.rename(columns = {'eGRID subregion name_x':'eGRID subregion name'}, inplace = True)

### Functions
Setting up the functions to be used in developing the emission factors.

#### Misc. Functions
Miscellanous functions used to develop the emission factors.

In [ ]:
#Summing up the different energy % types
def calculate_fossil_energy(coal, oil, gas, otherfossil, other, new_column, data):
  data[new_column] = data[coal]+data[oil]+data[gas]+data[otherfossil]+data[other]

def calculate_biomass_energy(biomass, new_column, data):
  data[new_column] = data[biomass]

def calculate_non_fossil_energy(nuclear, hydro, wind, solar, geothermal, new_column, data):
  data[new_column] = data[nuclear]+data[hydro]+data[wind]+data[solar]+data[geothermal]

In [ ]:
#Summing up the CO2e column
def calculate_co2e(co2, ch4, n2o, new_column, data):
  data[new_column] = data[co2]+data[ch4]+data[n2o]

#### Scope 2 Emission Factor Calculation Functions
Functions used to calculate emissions from electricity (scope 2).

In [ ]:
#Conversion of values to metric tons, application of GWPs for scope 2 factors
def calculate_scope2(column, new_column, GWP, data):
  emissions = data[column]
  my_list = []
  for i in emissions:
        my_list.append(i*0.00045359237*GWP)
  data[new_column] = my_list

In [ ]:
#Conversion of values to metric tons, putting emissions in CO2 versus biogenic CO2
def calculate_scope2_co2(column, new_column_co2, new_column_bioco2, fossilcolumn, biomasscolumn, data):
  emissions = data[column]
  biomass = data[biomasscolumn]
  fossil = data[fossilcolumn]
  my_list = []
  my_list_biogenic = []
  for i, v, x in zip(emissions, biomass, fossil):
      #If biomass is less than biomass%, use the fossil GWP
      if v < biomass_lower_limit:
        my_list.append(i*0.00045359237*co2_gwp)
        my_list_biogenic.append(0)
      #If fossil is less than fossil%, use the non-fossil GWP
      elif x < fossil_lower_limit:
        my_list.append(0)
        my_list_biogenic.append(i*0.00045359237*co2_gwp)
      #If fossil is less than fossil% but biomass is also less than fossil% use the fossil GWP
      elif x < fossil_lower_limit and v < fossil_lower_limit:
        my_list.append(i*0.00045359237*co2_gwp)
        my_list_biogenic.append(0)
      #Otherwise use the average GWP
      else:
        my_list.append(i*0.00045359237*co2_gwp)
        my_list_biogenic.append(0)
  data[new_column_co2] = my_list
  data[new_column_bioco2] = my_list_biogenic

In [ ]:
#Conversion of values to metric tons, application of GWPs, choosing GWPs for AR6 only
def calculate_scope2_ch4_ar6(column, new_column, fossilcolumn, biomasscolumn, data):
  emissions = data[column]
  biomass = data[biomasscolumn]
  fossil = data[fossilcolumn]
  my_list = []
  for i, v, x in zip(emissions, biomass, fossil):
      #If biomass is less than biomass%, use the fossil GWP
      if v < biomass_lower_limit:
        my_list.append(i*0.00045359237*ar6_ch4_gwp_fossil)
      #If fossil is less than fossil%, use the non-fossil GWP
      elif x < fossil_lower_limit:
        my_list.append(i*0.00045359237*ar6_ch4_gwp_nonfossil)
      #If fossil is less than fossil% but biomass is also less than fossil% use the fossil GWP
      elif x < fossil_lower_limit and v < fossil_lower_limit:
        my_list.append(i*0.00045359237*ar6_ch4_gwp_fossil)
      #Otherwise use the average GWP
      else:
        my_list.append(i*0.00045359237*ar6_ch4_gwp)
  data[new_column] = my_list

#### Scope 3 Emission Factor Calculation Functions
Functions used to calculate emissions from transmission and distribution (T&D) losses (scope 3).

In [ ]:
#Subregion: Conversion of values to metric tons, application of GWPs for T&D loss factors
def calculate_td(column, td_column, new_column, GWP, data):
  emissions = data[column]
  td_emissions = data[td_column]
  my_list = []
  for i, v in zip(emissions, td_emissions):
    my_list.append(i*0.00045359237*v/(1-v)*GWP)
  data[new_column] = my_list

In [ ]:
#Conversion of values to metric tons, application of GWPs for T&D loss factors, putting emissions in CO2 versus biogenic CO2
def calculate_td_co2(column, td_column, new_column_co2, new_column_biogenic, fossilcolumn, biomasscolumn, data):
  emissions = data[column]
  td_emissions = data[td_column]
  biomass = data[biomasscolumn]
  fossil = data[fossilcolumn]
  my_list = []
  my_list_biogenic = []
  for i, z, v, x in zip(emissions, td_emissions, biomass, fossil):
    #If biomass is less than biomass%, use the fossil GWP
    if v < biomass_lower_limit:
      my_list.append(i*0.00045359237*co2_gwp*z/(1-z))
      my_list_biogenic.append(0)
    #If fossil is less than fossil%, use the non-fossil GWP
    elif x < fossil_lower_limit:
      my_list.append(0)
      my_list_biogenic.append(i*0.00045359237*co2_gwp*z/(1-z))
    #If fossil is less than fossil% but biomass is also less than fossil% use the fossil GWP
    elif x < fossil_lower_limit and v < fossil_lower_limit:
      my_list.append(i*0.00045359237*co2_gwp*z/(1-z))
      my_list_biogenic.append(0)
    #Otherwise use the average GWP
    else:
      my_list.append(i*0.00045359237*co2_gwp*z/(1-z))
      my_list_biogenic.append(0)
  data[new_column_co2] = my_list
  data[new_column_biogenic] = my_list_biogenic

In [ ]:
#Conversion of values to metric tons, application of GWPs for T&D loss factors
def calculate_td_ch4_ar6(column, td_column, new_column, fossilcolumn, biomasscolumn, data):
  emissions = data[column]
  td_emissions = data[td_column]
  biomass = data[biomasscolumn]
  fossil = data[fossilcolumn]
  my_list = []
  for i, z, v, x in zip(emissions, td_emissions, biomass, fossil):
    #If biomass is less than biomass%, use the fossil GWP
    if v < biomass_lower_limit:
      my_list.append(i*0.00045359237*ar6_ch4_gwp_fossil*z/(1-z))
    #If fossil is less than fossil%, use the non-fossil GWP
    elif x < fossil_lower_limit:
      my_list.append(i*0.00045359237*ar6_ch4_gwp_nonfossil*z/(1-z))
    #If fossil is less than fossil% but biomass is also less than fossil% use the fossil GWP
    elif x < fossil_lower_limit and v < fossil_lower_limit:
      my_list.append(i*0.00045359237*ar6_ch4_gwp_fossil*z/(1-z))
    #Otherwise use the average GWP
    else:
      my_list.append(i*0.00045359237*ar6_ch4_gwp*z/(1-z))
  data[new_column] = my_list

### Energy Types
Calculating the percentages of each energy type for each subregion and putting the totals into three categories: Fossil, Biomass, Other energy (No Emissions).

In [ ]:
#Calculating the % of fossil, biomass or other (non-emissions) energy
#Fossil
column = column01
calculate_fossil_energy("Coal", "Oil", "Gas", "Other Fossil", "Other unknown/ purchased fuel", column, df_scope2)
calculate_fossil_energy("Coal", "Oil", "Gas", "Other Fossil", "Other unknown/ purchased fuel", column, df_scope3)

#Biomass
column = column02
calculate_biomass_energy("Biomass", column, df_scope2)
calculate_biomass_energy("Biomass", column, df_scope3)

#Other energy (No Emissions)
column = column03
calculate_non_fossil_energy("Nuclear", "Hydro", "Wind", "Solar", "Geo- thermal", column, df_scope2)
calculate_non_fossil_energy("Nuclear", "Hydro", "Wind", "Solar", "Geo- thermal", column, df_scope3)

### Emission Factors
Calculating emission factors for scope 2 and 3 and performing a union to add them one dataframe.

In [ ]:
#Assigning the scope number for the emission factors
column = column04
df_scope2[column] = 2
df_scope3[column] = 3

In [ ]:
#Calculating the emission factors with GWPs applied for scope 2
co2_column = column05
bioco2_column = column09
ch4_column = column06
n2o_column = column07
calculate_scope2_co2("CO2 - Total output emission rates", co2_column, bioco2_column, column01, column02, df_scope2)
calculate_scope2_ch4_ar6("CH4 - Total output emission rates", ch4_column, column01, column02, df_scope2)
calculate_scope2("N2O - Total output emission rates", n2o_column, ar6_n2o_gwp, df_scope2)

In [ ]:
#Calculating the emission factors with GWPs applied for scope 3
calculate_td_co2("CO2 - Total output emission rates", "Grid Gross Loss (%)", co2_column, bioco2_column, column01, column02, df_scope3)
calculate_td_ch4_ar6("CH4 - Total output emission rates", "Grid Gross Loss (%)", ch4_column, column01, column02, df_scope3)
calculate_td("N2O - Total output emission rates", "Grid Gross Loss (%)", n2o_column, ar6_n2o_gwp, df_scope3)

In [ ]:
#Summing CO2, CH4 and N2O into total CO2e
calculate_co2e(co2_column, ch4_column, n2o_column, column08, df_scope2)
calculate_co2e(co2_column, ch4_column, n2o_column, column08, df_scope3)

In [ ]:
#Dropping any rows if the CO2e is null (e.g., Puerto Rico didn't have a grid loss % in eGRID2021 so a factor can't be calculated)
column = column08
df_scope2 = df_scope2.dropna(subset=[column])
df_scope3 = df_scope3.dropna(subset=[column])

In [ ]:
#Union of the scope 2 and 3 emission factor dataframes
df_emission_factors = pd.concat([df_scope2, df_scope3])

In [ ]:
#Previewing the emission factor dataframe
df_emission_factors.head()

In [ ]:
#Using iloc to only view the columns we need
df_ef_final = df_emission_factors.iloc[:, [0,20,21,22,23,24,25]]

#Viewing the final emission factor data to be used in the calculations
df_ef_final

# Zip Codes for Subregions
Using EPA's Power Profiler data to search for the subregion by zip code. This will be used to look up the emission factor for the specific zip code (i.e., a zip code corresponds to a subregion and the emission factor for that subregion will be used).

In [ ]:
#Loading the zip code and subregion data
zip_codes = pd.read_excel('https://www.epa.gov/system/files/documents/2023-02/power_profiler_zipcode_tool.xlsx', sheet_name='Zip-subregion')

In [ ]:
#Previewing the zip code and subregion data
zip_codes.head()

In [ ]:
#Looking up the subregion based on the zip code that was entered
zip_data = zip_codes.loc[zip_codes['zip'] == user_zip, 'Subregion 1'].values[0]

In [ ]:
#Calculating emissions for each GHG and scope category
class EmissionCalculator:
    def __init__(self, df_ef_final, zip_data, energy_amount):
        self.df_ef_final = df_ef_final
        self.zip_data = zip_data
        self.energy_amount = energy_amount

    def calculate_co2e(self, scope):
        co2e_e_factor = self.df_ef_final.loc[(self.df_ef_final['scope'] == scope) & (self.df_ef_final['eGRID subregion acronym'] == self.zip_data), 'co2e_factor'].values[0]
        co2e = (self.energy_amount * co2e_e_factor).round(3)
        return co2e

    def calculate_co2(self, scope):
        co2_e_factor = self.df_ef_final.loc[(self.df_ef_final['scope'] == scope) & (self.df_ef_final['eGRID subregion acronym'] == self.zip_data), 'co2_factor'].values[0]
        co2 = (self.energy_amount * co2_e_factor).round(3)
        return co2

    def calculate_ch4(self, scope):
        ch4_e_factor = self.df_ef_final.loc[(self.df_ef_final['scope'] == scope) & (self.df_ef_final['eGRID subregion acronym'] == self.zip_data), 'ch4_factor'].values[0]
        ch4 = (self.energy_amount * ch4_e_factor).round(3)
        return ch4
    
    def calculate_n2o(self, scope):
        n2o_e_factor = self.df_ef_final.loc[(self.df_ef_final['scope'] == scope) & (self.df_ef_final['eGRID subregion acronym'] == self.zip_data), 'n2o_factor'].values[0]
        n2o = (self.energy_amount * n2o_e_factor).round(3)
        return n2o
    
    def calculate_bio_co2(self, scope):
        bio_co2_e_factor = self.df_ef_final.loc[(self.df_ef_final['scope'] == scope) & (self.df_ef_final['eGRID subregion acronym'] == self.zip_data), 'biogenic_co2_factor'].values[0]
        bio_co2 = (self.energy_amount * bio_co2_e_factor).round(3)
        return bio_co2

In [ ]:
#Creating an instance of EmissionCalculator
emission_calc = EmissionCalculator(df_ef_final, zip_data, energy_amount)

#Calculating emissions for scope 2 and 3
co2e_scope2 = emission_calc.calculate_co2e(2)
co2e_scope3 = emission_calc.calculate_co2e(3)

co2_scope2 = emission_calc.calculate_co2(2)
co2_scope3 = emission_calc.calculate_co2(3)

ch4_scope2 = emission_calc.calculate_ch4(2)
ch4_scope3 = emission_calc.calculate_ch4(3)

n2o_scope2 = emission_calc.calculate_n2o(2)
n2o_scope3 = emission_calc.calculate_n2o(3)

bio_co2_scope2 = emission_calc.calculate_bio_co2(2)
bio_co2_scope3 = emission_calc.calculate_bio_co2(3)

#Creating a table with the results
table_data = [
    ['CO2', co2_scope2, co2_scope3],
    ['Biogenic CO2', bio_co2_scope2, bio_co2_scope3],
    ['CH4', ch4_scope2, ch4_scope3],
    ['N2O', n2o_scope2, n2o_scope3], 
    ['Total CO2e', co2e_scope2, co2e_scope3]
]

table_headers = ['GHG Type', 'Scope 2', 'Scope 3']
table = tabulate(table_data, headers=table_headers, tablefmt='grid')

#Printing the final table
print("You consumed " + str(energy_amount) + " MWh of energy in your area (zip code " + str(user_zip) + "), which resulted in the following emissions (in metric tons CO2e):")
print(table)